In [1]:
import pandas as pd
import numpy as np

### Exemplo

Uma empresa de energéticos utiliza em sua fórmula
duas substâncias compradas de laboratórios terceirizados - ativina e baratol - os quais contêm dois dos principais ingredientes ativos da bebida: extrato de guaraná e taurina.
A fim de otimizar o custo de produção, a empresa deseja saber quantas doses de de cada ingrediente deve incluir em cada lata da bebida, para atingir as quantidades mínimas de 48 gramas por lata de extrato de guaraná e 12 gramas de taurina. Pelos seus severos
efeitos colaterais, a vigilância sanitária exige que a quantidade de taurina por lata não deve ultrapassar o limite de 20 gramas. Sabendo que cada dose de ativina custa 0.06 reais e contribui com 8 gramas de extrato de guaraná e 1 grama de taurina, enquanto que cada dose de baratol custa 0.08 reais contribui com 6 gramas de extrato de guaraná e 2 gramas de taurina.

- Função Objetivo:
  - Min Z = 0,06x1 + 0,08x2
- Restrições:
  - 1) Guaraná: 8a + 6b >= 48
  - 2) Taurina Min: 1a + 2b >= 12
  - 3) Taurina Máx: 1a + 2b <= 20
- Tal que:
  x1, x2 >=0

In [ ]:
obj = {'type': 'min', 'coeffs': [0.06, 0.08]}
constraints = [([8, 6]), ([1, 2]), ([1, 2])]
b_values = [48, 12, 20]
const_signs = ['>=', '>=', '<=']
vars_sign = ['non-negative', 'non-negative']

In [ ]:
# Gerando a forma canonica

def to_canonical_form(obj, const_coeffs, const_signs, b_values, vars_sign):
    if obj['type'].lower() == 'min':
        obj['coeffs'] = [-c for c in obj['coeffs']]
        obj['type'] = 'max'

    new_constraints = []
    for constr, s, b in zip(const_coeffs, const_signs, b_values):
        if s == '=':
            # Para restrições de igualdade, adicionamos duas desigualdades
            new_constraints.append(([-c for c in constr], -b))
            new_constraints.append((constr, b))
        elif s == '>=':
            # Para restrições de '>=' convertemos para '<='
            new_constraints.append(([-c for c in constr], -b))
        else:
            # Para restrições '<=' mantemos como estão
            new_constraints.append((constr, b))

    return obj, new_constraints


In [ ]:
canonical_obj, canonical_constraints = to_canonical_form(obj, constraints, const_signs, b_values, vars_sign)

In [ ]:
print(canonical_constraints)
print(canonical_obj)

[([-8, -6], -48), ([-1, -2], -12), ([1, 2], 20)]
{'type': 'max', 'coeffs': [-0.06, -0.08]}


In [ ]:
# Gerando a forma dual

def generate_dual(obj, constraints):
    # Função objetivo dual
    dual_obj = {'type': 'min', 'coeffs': [b for _, b in constraints]}

    # Restrições dual
    dual_constraints = []
    for i in range(len(obj['coeffs'])):
        # Coletando a i-ésima coluna das restrições primal para formar a i-ésima restrição dual
        dual_constr = [constr[i] for constr, _ in constraints]
        dual_constraints.append((dual_constr, '>=', obj['coeffs'][i]))

    return dual_obj, dual_constraints

In [ ]:
dual_obj, dual_constraints = generate_dual(canonical_obj, canonical_constraints)

In [ ]:
print(dual_obj)
print(dual_constraints)

{'type': 'min', 'coeffs': [-48, -12, 20]}
[([-8, -1, 1], '>=', -0.06), ([-6, -2, 2], '>=', -0.08)]
